In [1]:
import requests
from dotenv import load_dotenv
import os

load_dotenv()

TOKEN = os.getenv('TOKEN')

h = {"Authorization": f"Bearer {TOKEN}"}

# url = "https://api.github.com/graphql/"

url = "https://api.github.com/repos/OpenRA/OpenRA/issues"

# q = '{query { viewer {login} }'

try:
    r = requests.get(url, headers=h)
    print(r.status_code)
    print(r.json())
except Exception as e:
    print(e)


200
[{'url': 'https://api.github.com/repos/OpenRA/OpenRA/issues/21976', 'repository_url': 'https://api.github.com/repos/OpenRA/OpenRA', 'labels_url': 'https://api.github.com/repos/OpenRA/OpenRA/issues/21976/labels{/name}', 'comments_url': 'https://api.github.com/repos/OpenRA/OpenRA/issues/21976/comments', 'events_url': 'https://api.github.com/repos/OpenRA/OpenRA/issues/21976/events', 'html_url': 'https://github.com/OpenRA/OpenRA/pull/21976', 'id': 3242198074, 'node_id': 'PR_kwDOAA6lpM6fhV8I', 'number': 21976, 'title': 'Fixed bug where infinite production queue does not notify on ready', 'user': {'login': 'ivanpmartell', 'id': 1517777, 'node_id': 'MDQ6VXNlcjE1MTc3Nzc=', 'avatar_url': 'https://avatars.githubusercontent.com/u/1517777?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/ivanpmartell', 'html_url': 'https://github.com/ivanpmartell', 'followers_url': 'https://api.github.com/users/ivanpmartell/followers', 'following_url': 'https://api.github.com/users/ivanpmartell/fol